# Environment

In [54]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Prepare the data

In [55]:
import gdown
import os

# Download URL, the shakespeare.txt
url = f'https://drive.google.com/uc?id=1O4PZ8wOpp6yecoy8tMuVEIFS7XgyRJy9'

data_path = '../data'
text_path = f'{data_path}/shakespeare.txt'

if not os.path.exists(data_path):
    os.makedirs(data_path)

if not os.path.exists(text_path):
  gdown.download(url, text_path, quiet=False)


In [56]:
import re

with open(text_path) as f:
  text = f.read()
  
text = re.sub(r'\d+', '', text)
text = re.sub(r' +', ' ', text)

print(f"lenth of the text {len(text)}")

lenth of the text 5046489


In [57]:
print(text[:1000])

 From fairest creatures we desire increase,
 That thereby beauty's rose might never die,
 But as the riper should by time decease,
 His tender heir might bear his memory:
 But thou contracted to thine own bright eyes,
 Feed'st thy light's flame with self-substantial fuel,
 Making a famine where abundance lies,
 Thy self thy foe, to thy sweet self too cruel:
 Thou that art now the world's fresh ornament,
 And only herald to the gaudy spring,
 Within thine own bud buriest thy content,
 And tender churl mak'st waste in niggarding:
 Pity the world, or else this glutton be,
 To eat the world's due, by the grave and thee.


 
 When forty winters shall besiege thy brow,
 And dig deep trenches in thy beauty's field,
 Thy youth's proud livery so gazed on now,
 Will be a tattered weed of small worth held:
 Then being asked, where all thy beauty lies,
 Where all the treasure of thy lusty days;
 To say within thine own deep sunken eyes,
 Were an all-eating shame, and thriftless praise.
 How much m

In [58]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f"vocab size : {len(chars)}")
print("".join(chars))

vocab size : 74

 !"&'(),-.:;<>?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_`abcdefghijklmnopqrstuvwxyz|}


# Prepare tokenizer

In [59]:
# import tiktoken
# tokenizer = tiktoken.get_encoding('gpt2')
# tokens = tokenizer.encode(text)
# print(f"total tokens {len(tokens)}")
# print("decode result of \"hello world.\"", tokenizer.decode([31373, 995]))

In [60]:
class SimpleTokenizer:
  def __init__(self, text):
    self.chars = sorted(list(set(text)))
    self.token2id = {c : i for i, c in enumerate(chars)}
    self.id2token = {i : c for i, c in enumerate(chars)}
    
  def encode(self, text):
    return [self.token2id[c] for c in text]
  
  def decode(self, token_ids):
    return "".join([self.id2token[token_id] for token_id in token_ids])
  

In [61]:
tokenizer = SimpleTokenizer(text)
vocab_size = len(tokenizer.chars)
print(
  tokenizer.encode("Hello"),
  tokenizer.decode([23, 50, 57, 57, 60]),
  sep='\n')

[23, 50, 57, 57, 60]
Hello


# Prepare Data for torch

In [62]:
import torch

data = torch.tensor(tokenizer.encode(text), dtype = torch.long,
                    device=device) # torch.long can be used as index directly
print(data, data.shape, data.dtype)


tensor([ 1, 21, 63,  ..., 29, 19,  0], device='cuda:0') torch.Size([5046489]) torch.int64


In [63]:
train_data_size = int(data.shape[0] * 0.9)
train_data = data[:train_data_size].detach()
val_data = data[train_data_size:].detach()

In [64]:
from torch.utils.data import Dataset, DataLoader
class SimpleDataset(Dataset):
  def __init__(self, data, block_size = 8):
    self.data = data
    self.block_size = block_size
    
  def __len__(self):
    return len(self.data) - self.block_size

  def __getitem__(self, idx):
    x = self.data[idx: idx + self.block_size]
    y = self.data[idx + self.block_size]
    return x, y

In [65]:
BATCH_SIZE = 32
train_dataset = SimpleDataset(train_data)
val_dataset = SimpleDataset(val_data)

In [66]:
print(len(train_dataset), len(val_dataset))

4541832 504641


In [67]:
data[:9]

tensor([ 1, 21, 63, 60, 58,  1, 51, 46, 54], device='cuda:0')

In [68]:
train_dataset[0]

(tensor([ 1, 21, 63, 60, 58,  1, 51, 46], device='cuda:0'),
 tensor(54, device='cuda:0'))

In [69]:
import numpy as np
class SimpleDataloader(DataLoader):
  def __init__(self, dataset, batch_size=4, shuffle=True, **kwargs):
    super().__init__(dataset, batch_size=batch_size, shuffle=shuffle, **kwargs)
    self.shuffle = shuffle
    
  def __iter__(self):
    dataset_size = len(self.dataset)
    indices = np.arange(dataset_size)
    if self.shuffle:
        np.random.shuffle(indices)

    for start_idx in range(0, dataset_size - self.batch_size + 1, self.batch_size):
        batch_indices = indices[start_idx:start_idx + self.batch_size]
        yield (torch.stack([self.dataset[i][0] for i in batch_indices]).to(device),
              torch.stack([self.dataset[i][1] for i in batch_indices]).to(device))

In [70]:
# torch.manual_seed(0)
train_dataloader = SimpleDataloader(train_dataset, BATCH_SIZE)
val_dataloader = SimpleDataloader(val_dataset, BATCH_SIZE)

In [71]:
for i, batch in enumerate(train_dataloader):
  print(batch)
  if i == 1:
    break

(tensor([[63,  1, 68, 60, 63, 49, 64,  1],
        [65, 53, 50,  1, 60, 46, 56,  8],
        [68, 54, 57, 57,  1, 70, 60, 66],
        [57, 54, 52, 53, 65, 64,  1, 60],
        [57, 49,  1, 59, 60, 65,  1, 64],
        [57, 70,  1, 50, 59, 49, 50, 46],
        [63, 70,  1, 53, 50, 46, 63, 65],
        [70,  8,  1, 59, 46, 70,  8,  1],
        [59, 49,  1, 47, 60, 60, 65,  2],
        [64,  1, 58, 50, 12,  1, 53, 50],
        [58, 54, 59, 50,  1, 50, 70, 50],
        [16, 36, 29, 18, 20, 27, 30, 35],
        [46, 52, 50, 59, 50, 65,  8,  1],
        [47, 57, 50,  1, 47, 46, 49, 59],
        [51, 60, 63,  1, 57, 50, 46, 49],
        [46,  1, 58, 46, 59,  1, 65, 60],
        [50,  1, 48, 53, 46, 65, 10,  0],
        [34, 35, 20, 33, 10,  1, 23, 60],
        [56,  1, 57, 54, 56, 50, 68, 54],
        [40,  1, 38, 30, 33, 27, 19,  1],
        [61, 63, 46, 70,  1, 70, 60, 66],
        [50,  1, 53, 46, 65, 53,  1, 64],
        [50, 59, 65, 60, 58, 47,  5, 49],
        [64, 65,  7,  1, 53, 60, 

# Transformer GPT

In [72]:
# Model Configuration
VOCAB_SIZE = 74 # Should be set according to the tokenizer
EMBED_DIM = 12

In [73]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleGPT(nn.Module):
  def __init__(self):
    super().__init__()
    # (B, T)
    self.embedding = nn.Embedding(VOCAB_SIZE, VOCAB_SIZE) #(B, T, T)
    

  def forward(self, x, targets = None):
    """
    x should be in the form of (B, T)
    """
    logits = self.embedding(x)
    y = logits[:, -1,:]
    y = F.softmax(y, 1)

    if targets is None:
      loss = None
    else:
      targets_one_hot = F.one_hot(targets, VOCAB_SIZE).float()
      loss = F.cross_entropy(y, targets_one_hot)

    return y, loss
  
  def generate(self, x, max_tokens = 30):
    """
    x (B, T)

    ### returns
    y (B, T + max_tokens)

    ### Warning
    Because we do not have an EOF, so it will generate max_tokens actually
    """
    for _ in range(max_tokens):
      # x shape (B, T)
      probs, loss = self.forward(x) # y shape (B, vocab_size)
      y = torch.multinomial(probs, num_samples=1)
      x = torch.cat([x, y], dim = 1)

    return x

In [74]:
gpt = SimpleGPT().to(device)

# Training and Test

In [75]:
torch.manual_seed(0)

In [76]:
def mubble(max_tokens = 200):
  input = torch.zeros(1, 1, dtype=torch.long,device=device)
  output = gpt.generate(input, max_tokens)
  text = tokenizer.decode(output[0].tolist())
  return text

mubble()

'\nM\nnWR!FYcaB}?x ?P!Mb cqIl,nbI},U"QQb:x[RnKY}.bd]n,MgKy,t_Rr"\'U`|zRqZGxC"GYwM\'_HdUl[!d hEwVJ;i`VAF e;F;FTtVfgbI[n|}A?gaYnEvNm}msyku|r<Htc IyP,?(,HtVFP]vDx??\n`pSto`MKZd>Hw>D!oPIK[iUfOmS&"eTj`oYSQ.N<_RFR'

In [77]:
optimizer = torch.optim.AdamW(gpt.parameters(), lr = 1e-3)

In [78]:
def epoch():
  for i, batch in enumerate(train_dataloader):
    X, targets = batch
    logits, loss = gpt.forward(X, targets)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i % 1000 == 0:
      print(loss)
    
    if i == 10000:
      break

In [79]:
epoch()

tensor(4.3069, device='cuda:0', grad_fn=<DivBackward1>)
tensor(4.2934, device='cuda:0', grad_fn=<DivBackward1>)
tensor(4.2760, device='cuda:0', grad_fn=<DivBackward1>)
tensor(4.2405, device='cuda:0', grad_fn=<DivBackward1>)
tensor(4.2337, device='cuda:0', grad_fn=<DivBackward1>)
tensor(4.1470, device='cuda:0', grad_fn=<DivBackward1>)
tensor(4.1357, device='cuda:0', grad_fn=<DivBackward1>)
tensor(4.0566, device='cuda:0', grad_fn=<DivBackward1>)
tensor(4.1399, device='cuda:0', grad_fn=<DivBackward1>)
tensor(4.0509, device='cuda:0', grad_fn=<DivBackward1>)
tensor(4.0919, device='cuda:0', grad_fn=<DivBackward1>)


In [80]:
print(mubble())


 the the the the the the the che the the the hepe the the the the he the thand the the the ous t thand the ou the the the the tor the the the the f t the than the thouris the s the the he the the the 
